In [21]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [22]:
def ex_tag(sid, page):
    ### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###
    
    ## 1.
    url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}"\
    "#&date=%2000:00:00&page={page}"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    a_tag = soup.find_all("a")
    
    ## 2.
    tag_lst = []
    for a in a_tag:
        if "href" in a.attrs:  # href가 있는것만 고르는 것
            if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                tag_lst.append(a["href"])
                
    return tag_lst

In [23]:
def re_tag(sid):
    ### 특정 분야의 100페이지까지의 뉴스의 링크를 수집하여 중복 제거한 리스트로 변환하는 함수 ###
    re_lst = []
    for i in tqdm(range(100)):
        lst = ex_tag(sid, i+1)
        re_lst.extend(lst)

    # 중복 제거
    re_set = set(re_lst)
    re_lst = list(re_set)
    
    return re_lst

In [24]:
all_hrefs = {}
sids = [i for i in range(100,106)]  # 분야 리스트

# 각 분야별로 링크 수집해서 딕셔너리에 저장
for sid in sids:
    sid_data = re_tag(sid)
    all_hrefs[sid] = sid_data

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [25]:
def art_crawl(all_hrefs, sid, index):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Args: 
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학]
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    art_dic = {}
    
    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    url = all_hrefs[sid][index]
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    
    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str
    
    return art_dic

In [26]:
# 모든 섹션의 데이터 수집 (제목, 날짜, 본문, section, url)
section_lst = [s for s in range(100, 106)]
artdic_lst = []

for section in tqdm(section_lst):
    for i in tqdm(range(len(all_hrefs[section]))):
        art_dic = art_crawl(all_hrefs, section, i)
        art_dic["section"] = section
        art_dic["url"] = all_hrefs[section][i]
        artdic_lst.append(art_dic)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/618 [00:00<?, ?it/s]

  0%|          | 0/636 [00:00<?, ?it/s]

  0%|          | 0/652 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

In [27]:
import pandas as pd

art_df = pd.DataFrame(artdic_lst)

In [28]:
art_df.to_csv("article_df.csv")

In [ ]:
# -*- coding: utf-8 -*-from bs4 import BeautifulSoupfrom datetime import datetimeimport requestsimport pandas as pdimport re# 각 크롤링 결과 저장하기 위한 리스트 선언title_text = []link_text = []source_text = []date_text = []contents_text = []result = ｛｝# 엑셀로 저장하기 위한 변수RESULT_PATH = 'D:/키워드 분석/네이버뉴스(01)/'  # 결과 저장할 경로now = datetime.now()  # 파일이름 현 시간으로 저장하기# ％％# 뉴스 수집을 위한 def입니다.# 날짜 정제화 함수def date_cleansing(test):    try:        # 지난 뉴스        # 머니투데이  10면1단  2018.11.05. 네이버뉴스   보내기        pattern = '\\\\d+.(\\\\d+).(\\\\d+).'  # 정규표현식        r = re.compile(pattern)        match = r.search(test).group(0)  # 2018.11.05.        date_text.append(match)    except AttributeError:        # 최근 뉴스        # 이데일리 1시간 전 네이버뉴스 보내기        pattern = '\\\\w* (\\\\d\\\\w*)'  # 정규표현식        r = re.compile(pattern)        match = r.search(test).group(1)        # print(match)        date_text.append(match)# 내용 정제화 함수def contents_cleansing(contents):    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '',                                      str(contents)).strip()  # 앞에 필요없는 부분 제거    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '',                                       first_cleansing_contents).strip()  # 뒤에 필요없는 부분 제거 (새끼 기사)    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()    contents_text.append(third_cleansing_contents)    # print(contents_text)def crawler(maxpage, query, sort, s_date, e_date):    s_from = s_date.replace(".","")    e_to = e_date.replace(".","")    page = 1    maxpage_t = (int(maxpage) - 1) * 10 + 1  # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지    while page <= maxpage_t:        url = "<https://search.naver.com/search.naver?where=news&query=>" + query + "&sort=" + sort + "&ds=" + s_date + "&de=" + e_date + "&nso=so％3Ar％2Cp％3Afrom" + s_from + "to" + e_to + "％2Ca％3A&start=" + str(page)        response = requests.get(url)        html = response.text        # 뷰티풀소프의 인자값 지정        soup = BeautifulSoup(html, 'html.parser')        # <a>태그에서 제목과 링크주소 추출        atags = soup.select('._sp_each_title')        for atag in atags:            title_text.append(atag.text)     # 제목            link_text.append(atag['href'])   # 링크주소        # 신문사 추출        source_lists = soup.select('._sp_each_source')        for source_list in source_lists:            source_text.append(source_list.text)  # 신문사        # 날짜 추출        date_lists = soup.select('.txt_inline')        for date_list in date_lists:            test = date_list.text            date_cleansing(test)  # 날짜 정제 함수사용        # 본문요약본        contents_lists = soup.select('ul.type01 dl')        for contents_list in contents_lists:            # print('==='*40)            # print(contents_list)            contents_cleansing(contents_list)  # 본문요약 정제화        # 모든 리스트 딕셔너리형태로 저장        result = ｛"date": date_text, "title": title_text, "source": source_text, "contents": contents_text, "link": link_text｝        print(page)        df = pd.DataFrame(result)  # df로 변환        page += 10    # 새로 만들 파일이름 지정    outputFileName = '％s-％s-％s  ％s시 ％s분 ％s초 merging.xlsx' ％ (now.year, now.month, now.day, now.hour, now.minute, now.second)    df.to_excel(RESULT_PATH + outputFileName, sheet_name='sheet1')def crawler_csv(maxpage, query, sort, s_date, e_date):  # 엑셀이 아닌 csv로 작성하기    s_from = s_date.replace(".", "")    e_to = e_date.replace(".", "")    page = 1    maxpage_t = (int(maxpage) - 1) * 10 + 1  # 11 = 2페이지 21 = 3페이지 31 = 4페이지...81 = 9페이지, 91 = 10페이지, 101 = 11페이지    while page <= maxpage_t:        url = "<https://search.naver.com/search.naver?where=news&query=>" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so％3Ar％2Cp％3Afrom" + s_from + "to" + e_to + "％2Ca％3A&start=" + str(page)        response = requests.get(url)        html = response.text        # 뷰티풀소프의 인자값 지정        soup = BeautifulSoup(html, 'html.parser')        # <a>태그에서 제목과 링크주소 추출        atags = soup.select('._sp_each_title')        for atag in atags:            title_text.append(atag.text)     # 제목            link_text.append(atag['href'])   # 링크주소        # 신문사 추출        source_lists = soup.select('._sp_each_source')        for source_list in source_lists:            source_text.append(source_list.text)    # 신문사        # 날짜 추출        date_lists = soup.select('.txt_inline')        for date_list in date_lists:            test = date_list.text            date_cleansing(test)  # 날짜 정제 함수사용        # 본문요약본        contents_lists = soup.select('ul.type01 dl')        for contents_list in contents_lists:            # print('==='*40)            # print(contents_list)            contents_cleansing(contents_list)  # 본문요약 정제화        # 모든 리스트 딕셔너리형태로 저장        result = ｛"date": date_text, "title": title_text, "source": source_text, "contents": contents_text, "link": link_text｝        print(page)        df = pd.DataFrame(result)  # df로 변환        page += 10    # 새로 만들 파일이름 지정    outputFileName = '％s-％s-％s  ％s시 ％s분 ％s초 merging.csv' ％ (now.year, now.month, now.day, now.hour, now.minute, now.second)    df.to_csv(RESULT_PATH + outputFileName, mode='a', sep="")def main_mod():    x = 0  # 연도별로 반복시키기 위해서(400줄 이상 네이버가 검색 결과를 제공하지 않기 때문에, 400줄 단위로 작업해야 하며, 연도를 나눠야 한다.)    while x < 20:        maxpage = 400        query = '"문화영향평가"'        sort = "0"        s_date = "20｛0:0>2｝.01.01".format(x)  # 2019.01.01        e_date = "20｛0:0>2｝.12.31".format(x)   # 2019.04.28        crawler_csv(maxpage, query, sort, s_date, e_date)        x += 1# ％％main_mod()# ％％# 경우에 따라 마지막 뉴스가 수백번 중복되는 경우가 발생하여 중복 제거가 필요import csvmy_columns = [1, 2, 3, 4, 5]with open('D:/키워드 분석/네이버뉴스(01)/contents.csv', 'r', encoding='utf-8') as f:    with open('D:/키워드 분석/네이버뉴스(01)/contents_mod.txt', 'w', encoding='utf-8') as nf:        filereader = csv.reader(f)        filewriter = csv.writer(nf)  # 헤더를 안 지우는 이유는 이미 여러번 들어가 있다.        for row_list in filereader:            row_list_output = []            for index_value in my_columns:                row_list_output.append(row_list[index_value])            filewriter.writerow(row_list_output)with open('D:/키워드 분석/네이버뉴스(01)/contents_mod.txt', 'r', encoding='utf-8') as f:    list_file = f.readlines()list_file = [line.rstrip('\\\\n') for line in list_file]new_list = list(set(list_file))  # 중복 뉴스기사 제거하기with open('D:/키워드 분석/네이버뉴스(01)/contents_mod.txt', 'w', encoding='utf-8') as nf:    for line in new_list:        nf.write(line + '\\\\n')# 헤더(date,title,source,contents,link)가 여러번 들어가 있으나, 중복 제거에서 한 번만 남는다. 위치는 어디인지 알 수 없다.# 날짜가 아닌, 1일 7일 5시간 형태로 나타나는 경우가 존재하며, 이 경우 데이터 오류가 아닌, 수집 시점에서 일주일이 지나지 않을 경우 네이버가 날짜를 표시하지 않아 생기는 문제.